In [ ]:
#Importing necesssary modules
from pylab import *
import pandas as pd

from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

#Universal encoding library
import tensorflow as tf
import tensorflow_hub as hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

#Data location prefix
from google.colab import drive
drive.mount('/content/drive')
prefix = '/content/drive/My Drive/EE5180/Project/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
data = pd.read_csv(prefix + "questions.csv")
data = data.drop(['id','qid1','qid2'],axis=1)

dataArray = column_stack((data.question1,data.question2,data.is_duplicate))
data.head()

,question1,question2,is_duplicate
0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [ ]:
'''#Preprocessiong
dataVectors = []
for w in dataArray:
  try:
    embeddings = embed(w[:2])
    emdeddings = concatenate(( array(embeddings[0]),array(embeddings[1]),array([w[2]]) )) #Cocatenating embeddings of two sentences
    dataVectors.append(embeddings)
  except :
    break
dataVectors = array(dataVectors)
dataVectors[:,-1] = 2*dataVectors[:,-1] - 1
train_data, test_data = train_test_split(dataVectors, test_size= 0.2)
labels_train = train_data[:,-1]; vectorData_train = train_data[:,:-1]
labels_test = test_data[:,-1]; vectorData_test = test_data[:,:-1]'''


In [ ]:
test_data, gg = train_test_split(dataArray, test_size= 0.9)
rest, train_data = train_test_split(gg, test_size= 0.5)
print(len(test_data),len(train_data))

40435 181958


# Universal encoding

In [ ]:
#Preprocessiong
vectorData_train = []; labels_train = []
for w in train_data:
  try:
    embeddings = embed(w[:2])
    emdeddings = concatenate( (array(embeddings[0]),array(embeddings[1])) ) #Cocatenating embeddings of two sentences
    vectorData_train.append(emdeddings) 
    labels_train.append(w[2])
  except :
    pass
vectorData_train = array(vectorData_train)
labels_train = array(labels_train)
labels_train = labels_train*2 - ones(len(labels_train))

vectorData_test = []; labels_test = []
for w in test_data:
  try:
    embeddings = embed(w[:2])
    emdeddings = concatenate((array(embeddings[0]),array(embeddings[1])))
    vectorData_test.append(emdeddings) 
    labels_test.append(w[2])
  except:
    pass
vectorData_test = array(vectorData_test)
labels_test = array(labels_test)
labels_test = labels_test*2 - ones(len(labels_test))

In [ ]:
print( 'Size of train data:'+ str(len(vectorData_train)))
print( 'Size of test data:'+ str(len(vectorData_test)))

Size of train data:181957
Size of test data:40434


## Logistic Regression

In [ ]:
#Logistic Regression
reg = LogisticRegression(max_iter=1000).fit(vectorData_train, labels_train)
train = reg.score(vectorData_train, labels_train)
test = reg.score(vectorData_test, labels_test)


In [ ]:
print( 'Train accuracy: '+ str(100*train) + '%')
print( 'Test accuracy: '+ str(100*test) + '%')

Train accuracy: 72.94745461839885%
Test accuracy: 72.58248009101251%


## Random Forest

In [ ]:
#Random forest
reg = RandomForestClassifier(max_depth = 20,random_state=0).fit(vectorData_train, labels_train)
train = reg.score(vectorData_train, labels_train)
test = reg.score(vectorData_test, labels_test)

print( 'Train accuracy: '+ str(100*train) + '%')
print( 'Test accuracy: '+ str(100*test) + '%')

Train accuracy: 98.58098341915948%
Test accuracy: 78.46614235544344%


## Bad Results

In [ ]:
#Polynomial logistic Regression
def transform(data,T):
  out = data
  for i in range(2,T+1):
    out = concatenate((out, data**i), axis=1)
  return out
T=5
reg = LogisticRegression(random_state=0,max_iter=10000).fit(transform(vectorData_train,T), labels_train)
train = reg.score(transform(vectorData_train,T), labels_train)
test = reg.score(transform(vectorData_test,T), labels_test)
print(train,test)

0.7300866571699905 0.7271145535163046


In [ ]:
#Adaboost
reg = AdaBoostClassifier(n_estimators=10, random_state=0).fit(vectorData_train, labels_train)
train = reg.score(vectorData_train, labels_train)
test = reg.score(vectorData_test, labels_test)
print(train,test)

0.6646980848369737 0.6642532412864118
